# The first attempt to recreate a KAN

In [62]:
import torch.nn as nn
import torch
import numpy as np
from spline import *
from utils import *

In [79]:
class KANLayer(nn.Module):

    def __init__(self, in_dim, out_dim, num, k=3, noise_scale=0.1, scale_base=1.0, grid_range=[-1, 1], device='cpu'):
        super(KANLayer, self).__init__()

        # self.func_matrix = nn.Parameter(torch.zeros(in_dim, out_dim))

        # square = lambda x: x**2

        # self.func_matrix[0,0] = square

        self.size = size = out_dim * in_dim
        self.out_dim = out_dim
        self.in_dim = in_dim
        self.num = num
        self.k = k

        self.grid = torch.linspace(grid_range[0], grid_range[1], steps=num + 1, device=device).repeat(size, 1)
        self.grid = torch.nn.Parameter(self.grid).requires_grad_(False)

        # NO IDEA WHAT THIS DOES
        noises = (torch.rand(size, self.grid.shape[1]) - 1 / 2) * noise_scale / num
        noises = noises.to(device)

        self.coef = torch.nn.Parameter(curve2coef(self.grid, noises, self.grid, k, device))

        # NOR THIS
        # if isinstance(scale_base, float):
        #     self.scale_base = torch.nn.Parameter(torch.ones(size, device=device) * scale_base).requires_grad_(sb_trainable) 


        self.mask = torch.nn.Parameter(torch.ones(size, device=device)).requires_grad_(False)
        # self.grid_eps = grid_eps
        self.weight_sharing = torch.arange(size)
        self.lock_counter = 0
        self.lock_id = torch.zeros(size)
        self.device = device


    def forward(self, x):
        batch = x.shape[0]
        # x: shape (batch, in_dim) => shape (size, batch) (size = out_dim * in_dim)
        # x = torch.einsum('ij,k->ikj', x, torch.ones(self.out_dim, device=self.device)).reshape(batch, self.size).permute(1, 0)

        return x


In [80]:
class KAN(nn.Module):
    
    def __init__(self, width=None, grid=3, seed=69):
        super(KAN, self).__init__()

        self.biases = []
        self.act_fun = []
        self.depth = len(width) - 1
        self.width = width

        for l in range(self.depth):
            kan_layer = KANLayer(width[l], width[l+1], grid)
            self.act_fun.append(kan_layer)

        print(self.act_fun[0][0,0](3))

        


In [81]:
# create dataset f(x,y) = exp(sin(pi*x)+y^2)
f = lambda x: torch.exp(torch.sin(torch.pi*x[:,[0]]) + x[:,[1]]**2)
dataset = create_dataset(f, n_var=2)
dataset['train_input'].shape, dataset['train_label'].shape

(torch.Size([1000, 2]), torch.Size([1000, 1]))

In [90]:
class MyKANLayer(nn.Module):

    def __init__(self, in_dim, out_dim, num, k=3, noise_scale=0.1, scale_base=1.0, grid_range=[-1, 1], device='cpu'):
        super(MyKANLayer, self).__init__()

        self.in_dim = in_dim
        self.out_dim = out_dim

        # self.func_matrix = nn.Parameter(torch.ones(in_dim, out_dim))

        def square(x):
            return 2**x
        
        self.act_func = square


    def forward(self, x):

        y = torch.zeros(self.out_dim)

        for i in range(self.in_dim):
            for j in range(self.out_dim):
                y[j] += self.act_func(x[i])


        return y


In [91]:
# kan = KAN([2, 5, 2])

layer = MyKANLayer(2, 5, 3)

out = layer.forward(dataset['train_input'][0])
out.shape

torch.Size([5])